In [77]:
import math
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
from io import StringIO
import scipy.stats as ss
import xlrd

def SABR(T,K,S0,sigma,alpha,beta,rho):  #T = T_m, time to maturity, All inputs are scalars
    
    S0K = S0*K
    lS0K = np.log(S0/K)
    
    z = (sigma/alpha)*((S0K)**((1-beta)/2))*(lS0K)
    x = np.log((np.sqrt(1-2*rho*z+z**2)+z-rho)/(1-rho))
    
    denom = 1+(((1-beta)*lS0K)**2)/24 + (((1-beta)*lS0K)**4)/1920
    
    numer = 1 + T*((((1-beta)*alpha)**2)/(24*(S0K**(1-beta))) +  \
    (rho*beta*sigma*alpha)/(4*(S0K**((1-beta)/2))) + ((sigma**2)*(2-3*(rho**2)))/24)
    
    imp_vol = (alpha*numer*(z/x))/(denom*(S0K**((1-beta)/2)))
    
    return (alpha*numer)/(denom*(S0K**((1-beta)/2))) if np.any(S0==K) else imp_vol


def d1(T,K,F,sigma):
    return (np.log(F/K) + (sigma**2 / 2) * T)/(sigma * np.sqrt(T))
 
def d2(T,K,F,sigma):
    return (np.log(F/K) - (sigma**2 / 2) * T) / (sigma * np.sqrt(T))
 
def Black76(T,K,F,sigma):
    return (F * ss.norm.cdf(d1(T,K,F,sigma)) - K * ss.norm.cdf(d2(T,K,F,sigma)))

In [78]:
# Data from Citi
OTM_2024 = """
-200 -100 -75 -50 -25 ATM 25 50 75 100 200
10Y1Y 5,1416 5,1819 5,2105 5,2465 5,2898 5,3399 5,3966 5,4596 5,5283 5,6025 5,9448
10Y2Y 5,1588 5,1864 5,2116 5,2442 5,2841 5,3311 5,3845 5,4444 5,5102 5,5815 5,9138
10Y3Y 5,0699 5,0806 5,1017 5,1304 5,1665 5,2101 5,2599 5,3166 5,3795 5,4480 5,7709
10Y5Y 4,9781 4,9597 4,9735 4,9950 5,0241 5,0614 5,1043 5,1549 5,2119 5,2750 5,5792
10Y7Y 4,9234 4,8694 4,8762 4,8916 4,9157 4,9531 4,9896 5,0388 5,0955 5,1594 5,4749
10Y10Y 4,8882 4,7796 4,7750 4,7806 4,7966 4,8315 4,8598 4,9065 4,9626 5,0275 5,3610
10Y12Y 4,7487 4,6379 4,6305 4,6334 4,6470 4,6781 4,7064 4,7517 4,8068 4,8709 5,2033
10Y15Y 4,6292 4,4963 4,4859 4,4862 4,4974 4,5247 4,5530 4,5970 4,6509 4,7143 5,0457
10Y20Y 4,4983 4,3478 4,3329 4,3288 4,3360 4,3559 4,3846 4,4256 4,4772 4,5385 4,8650
10Y30Y 4,3647 4,1915 4,1706 4,1605 4,1619 4,1757 4,2003 4,2370 4,2847 4,3428 4,6603
"""

OTM_2024 = OTM_2024.replace(",", ".")
OTM_2024_io = StringIO(OTM_2024)

df_OTM_2024 = pd.read_csv(OTM_2024_io, sep=" ")
df_OTM_2024 = df_OTM_2024 *10

df_OTM_2024



,-200,-100,-75,-50,-25,ATM,25,50,75,100,200
10Y1Y,51.416,51.819,52.105,52.465,52.898,53.399,53.966,54.596,55.283,56.025,59.448
10Y2Y,51.588,51.864,52.116,52.442,52.841,53.311,53.845,54.444,55.102,55.815,59.138
10Y3Y,50.699,50.806,51.017,51.304,51.665,52.101,52.599,53.166,53.795,54.480,57.709
10Y5Y,49.781,49.597,49.735,49.950,50.241,50.614,51.043,51.549,52.119,52.750,55.792
10Y7Y,49.234,48.694,48.762,48.916,49.157,49.531,49.896,50.388,50.955,51.594,54.749
10Y10Y,48.882,47.796,47.750,47.806,47.966,48.315,48.598,49.065,49.626,50.275,53.610
10Y12Y,47.487,46.379,46.305,46.334,46.470,46.781,47.064,47.517,48.068,48.709,52.033
10Y15Y,46.292,44.963,44.859,44.862,44.974,45.247,45.530,45.970,46.509,47.143,50.457
10Y20Y,44.983,43.478,43.329,43.288,43.360,43.559,43.846,44.256,44.772,45.385,48.650
10Y30Y,43.647,41.915,41.706,41.605,41.619,41.757,42.003,42.370,42.847,43.428,46.603


In [133]:
DataTable = df_OTM_2024.copy()
for col in DataTable.columns: 
        DataTable[col] = df_OTM_2024['ATM']/100 + df_OTM_2024[col]/10000
DataTable['ATM'] = df_OTM_2024['ATM']/100
DataTable

,-200,-100,-75,-50,-25,ATM,25,50,75,100,200
10Y1Y,0.539132,0.539172,0.539200,0.539237,0.539280,0.53399,0.539387,0.539450,0.539518,0.539592,0.539935
10Y2Y,0.538269,0.538296,0.538322,0.538354,0.538394,0.53311,0.538494,0.538554,0.538620,0.538691,0.539024
10Y3Y,0.526080,0.526091,0.526112,0.526140,0.526176,0.52101,0.526270,0.526327,0.526389,0.526458,0.526781
10Y5Y,0.511118,0.511100,0.511113,0.511135,0.511164,0.50614,0.511244,0.511295,0.511352,0.511415,0.511719
10Y7Y,0.500233,0.500179,0.500186,0.500202,0.500226,0.49531,0.500300,0.500349,0.500405,0.500469,0.500785
10Y10Y,0.488038,0.487930,0.487925,0.487931,0.487947,0.48315,0.488010,0.488056,0.488113,0.488177,0.488511
10Y12Y,0.472559,0.472448,0.472440,0.472443,0.472457,0.46781,0.472516,0.472562,0.472617,0.472681,0.473013
10Y15Y,0.457099,0.456966,0.456956,0.456956,0.456967,0.45247,0.457023,0.457067,0.457121,0.457184,0.457516
10Y20Y,0.440088,0.439938,0.439923,0.439919,0.439926,0.43559,0.439975,0.440016,0.440067,0.440128,0.440455
10Y30Y,0.421935,0.421762,0.421741,0.421731,0.421732,0.41757,0.421770,0.421807,0.421855,0.421913,0.422230


In [134]:
DataTable['-200']

10Y1Y     0.539132
10Y2Y     0.538269
10Y3Y     0.526080
10Y5Y     0.511118
10Y7Y     0.500233
10Y10Y    0.488038
10Y12Y    0.472559
10Y15Y    0.457099
10Y20Y    0.440088
10Y30Y    0.421935
Name: -200, dtype: float64

In [142]:
((np.log(DataTable['-200'].iloc[2])-np.log(DataTable['-200'].iloc[1]))/(3-2))


-0.022904961299351156

In [143]:
(np.log(DataTable['-200'].iloc[2])-np.log(DataTable['-200'].iloc[1]))/(3-2)

-0.022904961299351156

In [146]:
DataTable.loc['10Y10Y']

-200    0.488038
-100    0.487930
-75     0.487925
-50     0.487931
-25     0.487947
ATM     0.483150
25      0.488010
50      0.488056
75      0.488113
100     0.488177
200     0.488511
Name: 10Y10Y, dtype: float64

In [160]:
import numpy as np
from datetime import datetime, timedelta

Settle = '12-Jun-2013'
ExerciseDates = ['12-Sep-2013', '12-Jun-2014', '12-Jun-2015', '12-Jun-2016', '12-Jun-2017', '12-Jun-2018', '12-Jun-2020', '12-Jun-2023']

def yearfrac(start_date, end_dates, basis=1):
    """
    Calculate the year fraction between a start date and a list of end dates.
    
    Parameters:
    start_date (str): The start date in the format 'DD-MMM-YYYY'.
    end_dates (list): A list of end dates in the format 'DD-MMM-YYYY'.
    basis (int): The day count convention (0=Actual/Actual, 1=Actual/365, 2=Actual/360, 3=30/360, 4=Actual/Actual (ISDA), 5=Actual/365 (ISDA)).
    
    Returns:
    numpy.ndarray: The year fraction between the start date and each end date.
    """
    start_date = datetime.strptime(start_date, '%d-%b-%Y')
    end_dates = [datetime.strptime(date, '%d-%b-%Y') for date in end_dates]
    
    if basis == 0:
        return [(end_date - start_date).days / 365.25 for end_date in end_dates]
    elif basis == 1:
        return [(end_date - start_date).days / 365 for end_date in end_dates]
    elif basis == 2:
        return [(end_date - start_date).days / 360 for end_date in end_dates]
    elif basis == 3:
        return [((end_date.day - start_date.day) + (end_date.month - start_date.month) * 30 + (end_date.year - start_date.year) * 360) / 360 for end_date in end_dates]
    elif basis == 4:
        return [(end_date - start_date).days / 365 for end_date in end_dates]
    elif basis == 5:
        return [(end_date - start_date).days / 365 for end_date in end_dates]
    else:
        raise ValueError("Invalid basis value. Must be 0, 1, 2, 3, 4, or 5.")

YearsToExercise = yearfrac(Settle, ExerciseDates, 1)
NumMaturities = len(YearsToExercise)

MarketVolatilities = np.array([
   [57.6, 53.7, 49.4, 45.6, 44.1, 41.1, 35.2, 32.0],
   [46.6, 46.9, 44.8, 41.6, 39.8, 37.4, 33.4, 31.0],
   [35.9, 39.3, 39.6, 37.9, 37.2, 34.7, 30.5, 28.9],
   [34.1, 36.5, 37.8, 36.6, 35.0, 31.9, 28.1, 26.6],
   [41.0, 41.3, 39.5, 37.8, 36.0, 32.6, 29.0, 26.0],
   [45.8, 43.4, 41.9, 39.2, 36.9, 33.2, 29.6, 26.3],
   [50.3, 46.9, 44.0, 40.0, 37.5, 33.8, 30.2, 27.3]]) / 100

MarketStrikes = np.array([
  [1.00, 1.25, 1.68, 2.00, 2.26, 2.41, 2.58, 2.62],
  [1.50, 1.75, 2.18, 2.50, 2.76, 2.91, 3.08, 3.12],
  [2.00, 2.25, 2.68, 3.00, 3.26, 3.41, 3.58, 3.62],
  [2.50, 2.75, 3.18, 3.50, 3.76, 3.91, 4.08, 4.12],
  [3.00, 3.25, 3.68, 4.00, 4.26, 4.41, 4.58, 4.62],
  [3.50, 3.75, 4.18, 4.50, 4.76, 4.91, 5.08, 5.12],
  [4.00, 4.25, 4.68, 5.00, 5.26, 5.41, 5.58, 5.62]]) / 100

CurrentForwardValues = MarketStrikes[3, :]
CurrentForwardValues

array([0.025 , 0.0275, 0.0318, 0.035 , 0.0376, 0.0391, 0.0408, 0.0412])

In [161]:
ATMVolatilities = MarketVolatilities[3, :]
ATMVolatilities

array([0.341, 0.365, 0.378, 0.366, 0.35 , 0.319, 0.281, 0.266])

In [164]:
import numpy as np
from scipy.optimize import lsqnonlin, roots
from scipy.special import blackvolatility as blackvolbysabr

Beta = 0.5
Betas = np.repeat(Beta, NumMaturities)
Alphas = np.zeros(NumMaturities)
Rhos = np.zeros(NumMaturities)
Nus = np.zeros(NumMaturities)

options = {'display': 'none'}

for k in range(NumMaturities):
    # This function solves the SABR at-the-money volatility equation as a
    # polynomial of Alpha
    def alpharoots(Rho, Nu):
        return roots([
            (1 - Beta)**2 * YearsToExercise[k] / 24 / CurrentForwardValues[k]**(2 - 2*Beta),
            Rho * Beta * Nu * YearsToExercise[k] / 4 / CurrentForwardValues[k]**(1 - Beta),
            1 + (2 - 3*Rho**2) * Nu**2 * YearsToExercise[k] / 24,
            -ATMVolatilities[k] * CurrentForwardValues[k]**(1 - Beta)
        ])

    # This function converts at-the-money volatility into Alpha by picking the
    # smallest positive real root
    def atmVol2SabrAlpha(Rho, Nu):
        roots_arr = alpharoots(Rho, Nu)
        return min([x for x in roots_arr if x > 0] or [np.real(x) for x in roots_arr if np.abs(np.imag(x)) <= 1e-6])

    # Fit Rho and Nu (while converting at-the-money volatility into Alpha)
    def objFun(X):
        Rho, Nu = X
        return MarketVolatilities[:, k] - blackvolbysabr(atmVol2SabrAlpha(Rho, Nu), Beta, Rho, Nu, Settle, ExerciseDates[k], CurrentForwardValues[k], MarketStrikes[:, k])

    X = lsqnonlin(objFun, [0, 0.5], [-1, 0], [1, np.inf], options)
    Rho, Nu = X
    
    # Get final Alpha from the calibrated parameters
    Alpha = atmVol2SabrAlpha(Rho, Nu)

    Alphas[k] = Alpha
    Rhos[k] = Rho
    Nus[k] = Nu

CalibratedPrameters = np.column_stack((Alphas, Betas, Rhos, Nus))
CalibratedPrameters = pd.DataFrame(CalibratedPrameters, index=['3M into 10Y', '1Y into 10Y', '2Y into 10Y', '3Y into 10Y', '4Y into 10Y', '5Y into 10Y', '7Y into 10Y', '10Y into 10Y'], columns=['Alpha', 'Beta', 'Rho', 'Nu'])



ImportError: cannot import name 'lsqnonlin' from 'scipy.optimize' (/Users/nannaingemannohrt/anaconda3/envs/master_thesis_venv/lib/python3.10/site-packages/scipy/optimize/__init__.py)

In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import least_squares

# Placeholder for the actual Black volatility calculation by SABR model
def blackvolbysabr(alpha, beta, rho, nu, settle, exercise_date, fwd_value, strike):
    return np.log(alpha / fwd_value)  # Simplified placeholder

# Hypothetical data setup
NumMaturities = 8
YearsToExercise = np.array([1, 2, 3, 4, 5, 6, 7, 8])
CurrentForwardValues = np.linspace(100, 102, NumMaturities)
ATMVolatilities = np.linspace(0.2, 0.22, NumMaturities)
MarketVolatilities = np.random.normal(0.05, 0.06, (10, NumMaturities))
Settle = np.datetime64('today')
ExerciseDates = np.array([np.datetime64('today') + np.timedelta64(i, 'D') for i in range(NumMaturities)])
MarketStrikes = np.random.uniform(100, 200, (10, NumMaturities))

Beta = 0.5
Betas = np.repeat(Beta, NumMaturities)
Alphas = np.zeros(NumMaturities)
Rhos = np.zeros(NumMaturities)
Nus = np.zeros(NumMaturities)

for k in range(NumMaturities):
    def alpharoots(rho, nu):
        coefficients = [
            (1 - Beta)**2 * YearsToExercise[k] / 24 / CurrentForwardValues[k]**(2 - 2 * Beta),
            rho * Beta * nu * YearsToExercise[k] / 4 / CurrentForwardValues[k]**(1 - Beta),
            1 + (2 - 3 * rho**2) * nu**2 * YearsToExercise[k] / 24,
            -ATMVolatilities[k] * CurrentForwardValues[k]**(1 - Beta)
        ]
        return np.roots(coefficients)

    def atmVol2SabrAlpha(rho, nu):
        roots_arr = alpharoots(rho, nu)
        real_roots = [x for x in roots_arr if x.imag == 0 and x.real > 0]
        if real_roots:
            return min(real_roots).real
        return min(roots_arr, key=abs).real

    def objFun(X):
        rho, nu = X
        alpha = atmVol2SabrAlpha(rho, nu)
        return MarketVolatilities[:, k] - blackvolbysabr(alpha, Beta, rho, nu, Settle, ExerciseDates[k], CurrentForwardValues[k], MarketStrikes[:, k])

    result = least_squares(objFun, [0, 0.5], bounds=([-1, 0], [1, np.inf]), verbose=0)
    rho, nu = result.x
    Alpha = atmVol2SabrAlpha(rho, nu)
    Alphas[k] = Alpha
    Rhos[k] = rho
    Nus[k] = nu

# Create a DataFrame to store the calibrated parameters
CalibratedParameters = np.column_stack((Alphas, Betas, Rhos, Nus))
CalibratedParameters = pd.DataFrame(CalibratedParameters, index=['3M into 10Y', '1Y into 10Y', '2Y into 10Y', '3Y into 10Y', '4Y into 10Y', '5Y into 10Y', '7Y into 10Y', '10Y into 10Y'], columns=['Alpha', 'Beta', 'Rho', 'Nu'])
CalibratedParameters

,Alpha,Beta,Rho,Nu
3M into 10Y,621.011463,0.5,-0.851244,6.188318
1Y into 10Y,310.786845,0.5,-0.885586,3.025742
2Y into 10Y,295.221523,0.5,-0.838214,3.020700
3Y into 10Y,154.817983,0.5,-0.953904,1.446297
4Y into 10Y,145.238853,0.5,-0.901694,1.432669
5Y into 10Y,138.971669,0.5,-0.865171,1.425140
7Y into 10Y,134.756256,0.5,-0.838297,1.422106
10Y into 10Y,144.836155,0.5,-0.806296,1.570870


In [2]:
import pandas as pd

# Example of manually entering the data:
data = {
    'Tenor': ['1Y', '2Y', '3Y', '5Y', '7Y', '10Y', '12Y', '15Y', '20Y', '30Y'],
    '-200': [0.539132, 0.538269, 0.526080, 0.511118, 0.500233, 0.488038, 0.472559, 0.457099, 0.440088, 0.421935],
    '-100': [0.539172, 0.538296, 0.526091, 0.511100, 0.500179, 0.487930, 0.472448, 0.456966, 0.439938, 0.421762],
    '-75': [0.539200, 0.538322, 0.526112, 0.511113, 0.500186, 0.487925, 0.472440, 0.456956, 0.439923, 0.421731],
    '-50': [0.539237, 0.538354, 0.526140, 0.511135, 0.500202, 0.487931, 0.472483, 0.456956, 0.439919, 0.421731],
    '-25': [0.539280, 0.538394, 0.526176, 0.511144, 0.500226, 0.487947, 0.472457, 0.456967, 0.439926, 0.421732],
    'ATM': [0.539328, 0.533911, 0.521010, 0.506140, 0.495310, 0.483150, 0.467810, 0.452470, 0.435590, 0.417570],
    '25': [0.539387, 0.538404, 0.526270, 0.511195, 0.500300, 0.488010, 0.472516, 0.457023, 0.439975, 0.421770],
    '50': [0.539450, 0.538554, 0.526327, 0.511295, 0.500349, 0.488056, 0.472562, 0.457067, 0.440016, 0.421807],
    '75': [0.539518, 0.538620, 0.526389, 0.511352, 0.500405, 0.488113, 0.472617, 0.457184, 0.440067, 0.421855],
    '100': [0.539592, 0.538691, 0.526458, 0.511415, 0.500469, 0.488177, 0.472681, 0.457184, 0.440128, 0.421913],
    '200': [0.539935, 0.539024, 0.526781, 0.511719, 0.500785, 0.488811, 0.473013, 0.457516, 0.440455, 0.422230]
}

# Creating DataFrame
vol_data = pd.DataFrame(data)
vol_data.set_index('Tenor', inplace=True)

# Displaying the DataFrame to verify the data
print(vol_data)


           -200      -100       -75       -50       -25       ATM        25  \
Tenor                                                                         
1Y     0.539132  0.539172  0.539200  0.539237  0.539280  0.539328  0.539387   
2Y     0.538269  0.538296  0.538322  0.538354  0.538394  0.533911  0.538404   
3Y     0.526080  0.526091  0.526112  0.526140  0.526176  0.521010  0.526270   
5Y     0.511118  0.511100  0.511113  0.511135  0.511144  0.506140  0.511195   
7Y     0.500233  0.500179  0.500186  0.500202  0.500226  0.495310  0.500300   
10Y    0.488038  0.487930  0.487925  0.487931  0.487947  0.483150  0.488010   
12Y    0.472559  0.472448  0.472440  0.472483  0.472457  0.467810  0.472516   
15Y    0.457099  0.456966  0.456956  0.456956  0.456967  0.452470  0.457023   
20Y    0.440088  0.439938  0.439923  0.439919  0.439926  0.435590  0.439975   
30Y    0.421935  0.421762  0.421731  0.421731  0.421732  0.417570  0.421770   

             50        75       100       200  
Ten

In [3]:
NumMaturities = len(vol_data.index)
YearsToExercise = np.array([1, 2, 3, 5, 7, 10, 12, 15, 20, 30])  # Update based on the tenors
CurrentForwardValues = np.linspace(100, 200, NumMaturities)  # Define as needed or get from elsewhere
ATMVolatilities = vol_data['ATM'].values  # Use ATM volatilities directly from the data


In [4]:
import numpy as np
import pandas as pd
from scipy.optimize import least_squares

# Placeholder for the actual Black volatility calculation by SABR model
def blackvolbysabr(alpha, beta, rho, nu, settle, exercise_date, fwd_value, strike):
    # Placeholder function, should be replaced with the actual implementation
    return np.log(alpha / fwd_value)  # Simplified placeholder

# Define the data from the image
vol_data = {
    'Tenor': ['1Y', '2Y', '3Y', '5Y', '7Y', '10Y', '12Y', '15Y', '20Y', '30Y'],
    'ATM': [0.539328, 0.533911, 0.521010, 0.506140, 0.495310, 0.483150, 0.467810, 0.452470, 0.435590, 0.417570]
}
vol_df = pd.DataFrame(vol_data)
vol_df.set_index('Tenor', inplace=True)

# Variables setup
NumMaturities = len(vol_df)
YearsToExercise = np.array([1, 2, 3, 5, 7, 10, 12, 15, 20, 30])  # Years until exercise for each tenor
CurrentForwardValues = np.linspace(100, 200, NumMaturities)  # Simulated forward values
ATMVolatilities = vol_df['ATM'].values  # ATM volatilities from the data

Beta = 0.5
Betas = np.repeat(Beta, NumMaturities)
Alphas = np.zeros(NumMaturities)
Rhos = np.zeros(NumMaturities)
Nus = np.zeros(NumMaturities)

Settle = np.datetime64('today')
ExerciseDates = [Settle + np.timedelta64(int(year), 'Y') for year in YearsToExercise]

for k in range(NumMaturities):
    def alpharoots(rho, nu):
        coefficients = [
            (1 - Beta)**2 * YearsToExercise[k] / 24 / CurrentForwardValues[k]**(2 - 2 * Beta),
            rho * Beta * nu * YearsToExercise[k] / 4 / CurrentForwardValues[k]**(1 - Beta),
            1 + (2 - 3 * rho**2) * nu**2 * YearsToExercise[k] / 24,
            -ATMVolatilities[k] * CurrentForwardValues[k]**(1 - Beta)
        ]
        return np.roots(coefficients)

    def atmVol2SabrAlpha(rho, nu):
        roots_arr = alpharoots(rho, nu)
        real_roots = [x for x in roots_arr if x.imag == 0 and x.real > 0]
        if real_roots:
            return min(real_roots).real
        return min(roots_arr, key=abs).real

    def objFun(X):
        rho, nu = X
        alpha = atmVol2SabrAlpha(rho, nu)
        return [ATMVolatilities[k] - blackvolbysabr(alpha, Beta, rho, nu, Settle, ExerciseDates[k], CurrentForwardValues[k], CurrentForwardValues[k])]

    result = least_squares(objFun, [0, 0.5], bounds=([-1, 0], [1, np.inf]), verbose=0)
    rho, nu = result.x
    Alpha = atmVol2SabrAlpha(rho, nu)
    Alphas[k] = Alpha
    Rhos[k] = rho
    Nus[k] = nu

# Create a DataFrame to store the calibrated parameters
CalibratedParameters = np.column_stack((Alphas, Betas, Rhos, Nus))
CalibratedParameters = pd.DataFrame(CalibratedParameters, index=vol_df.index, columns=['Alpha', 'Beta', 'Rho', 'Nu'])
print(CalibratedParameters)


UFuncTypeError: Cannot cast ufunc 'add' input 1 from dtype('<m8[Y]') to dtype('<m8[D]') with casting rule 'same_kind'

In [ ]:
## new try

In [177]:
import numpy as np
import pandas as pd
from scipy.optimize import least_squares, root
from scipy.stats import norm

# Placeholder definitions (example values should be replaced with actual data)
NumMaturities = 5
YearsToExercise = np.array([1, 2, 3, 4, 5])
CurrentForwardValues = np.array([100, 101, 102, 103, 104])
ATMVolatilities = np.array([0.2, 0.21, 0.22, 0.23, 0.24])
MarketVolatilities = np.random.rand(3, NumMaturities)  # Placeholder for 3x5 matrix
Settle = 0
ExerciseDates = np.array([1, 2, 3, 4, 5])
MarketStrikes = np.random.rand(3, NumMaturities) * 100  # Placeholder for 3x5 matrix

Beta = 0.5
Betas = np.repeat(Beta, NumMaturities)
Alphas = np.zeros(NumMaturities)
Rhos = np.zeros(NumMaturities)
Nus = np.zeros(NumMaturities)

options = {'ftol': 1e-9, 'xtol': 1e-9, 'gtol': 1e-9, 'verbose': 0}

def sabr_volatility(F, K, T, alpha, beta, rho, nu):
    """
    Calculate the SABR model implied volatility.
    """
    z = nu / alpha * (F * K)**((1 - beta) / 2) * np.log(F / K)
    x = np.log((np.sqrt(1 - 2 * rho * z + z**2) + z - rho) / (1 - rho))
    return (alpha / ((F * K)**((1 - beta) / 2))) * (z / x) * (1 + (((1 - beta)**2 / 24) * alpha**2 / ((F * K)**(1 - beta)) + (rho * beta * nu * alpha) / (4 * (F * K)**((1 - beta) / 2)) + (2 - 3 * rho**2) * nu**2 / 24) * T)

for k in range(NumMaturities):
    def objFun(X):
        Rho, Nu = X
        Alpha = ATMVolatilities[k] * CurrentForwardValues[k]**(1 - Beta)  # Simplified Alpha computation
        return MarketVolatilities[:, k] - sabr_volatility(CurrentForwardValues[k], MarketStrikes[:, k], YearsToExercise[k], Alpha, Beta, Rho, Nu)

    result = least_squares(objFun, [0, 0.5], bounds=([-1, 0], [1, np.inf]), **options)
    Rho, Nu = result.x
    Alpha = ATMVolatilities[k] * CurrentForwardValues[k]**(1 - Beta)  # Final Alpha computation based on calibration
    
    Alphas[k] = Alpha
    Rhos[k] = Rho
    Nus[k] = Nu

CalibratedParameters = np.column_stack((Alphas, Betas, Rhos, Nus))
CalibratedParameters = pd.DataFrame(CalibratedParameters, index=['3M into 10Y', '1Y into 10Y', '2Y into 10Y', '3Y into 10Y', '4Y into 10Y'], columns=['Alpha', 'Beta', 'Rho', 'Nu'])
print(CalibratedParameters)


                Alpha  Beta       Rho         Nu
3M into 10Y  2.000000   0.5  0.820085  52.359927
1Y into 10Y  2.110474   0.5 -0.719398   0.787700
2Y into 10Y  2.221891   0.5 -0.814048  16.911044
3Y into 10Y  2.334245   0.5 -1.000000   0.128506
4Y into 10Y  2.447529   0.5  1.000000   0.242103


In [188]:
import numpy as np
from scipy.optimize import minimize

def sabr_vol(alpha, rho, nu, F, K, T, beta):
    if F == K:
        z = 0  # To handle division by zero when F = K
        x_z = 1  # log(1) = 0, hence simplified expression
    else:
        z = (nu / alpha) * (F * K)**((1 - beta) / 2) * np.log(F / K)
        x_z = np.log((np.sqrt(1 - 2 * rho * z + z**2) + z - rho) / (1 - rho))
    factor = (F * K)**((beta - 1) / 2)
    return (alpha / factor) * (z / x_z)

def objective(params, market_vols, strikes, maturity, forward, beta):
    alpha, rho, nu = params
    model_vols = [sabr_vol(alpha, rho, nu, forward, K, maturity, beta) for K in strikes]
    return np.sum((model_vols - market_vols) ** 2)

# Example market data
market_vols = np.array([0.2, 0.22, 0.25])  # Example volatilities
strikes = np.array([100, 110, 120])        # Corresponding strike prices
maturity = 10  
#forward = 100                            # Maturity in years
forward = np.array([105,105.1])                              # Example forward rate
beta_fixed = 0.5                           # Example fixed beta

# Initial guesses for parameters
initial_guess = [0.1, 0, 0.2]

# Boundaries for the parameters: alpha > 0, -1 < rho < 1, nu > 0
bounds = [(0, None), (-1, 1), (0, None)]

# Perform the minimization
result = minimize(objective, initial_guess, args=(market_vols, strikes, maturity, forward, beta_fixed), bounds=bounds)
alpha_est, rho_est, nu_est = result.x

print("Estimated Parameters:", alpha_est, rho_est, nu_est)


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [193]:
import numpy as np
from scipy.optimize import minimize

def sabr_vol(alpha, rho, nu, F, K, T, beta):
    # Handle the case where forward and strike are arrays
    if np.any(F == K):
        z = np.zeros_like(F)  # Avoid division by zero issues
        x_z = np.zeros_like(F) + 1  # log(1) = 0, so x(z) simplifies
    else:
        z = (nu / alpha) * (F * K)**((1 - beta) / 2) * np.log(F / K)
        x_z = np.log((np.sqrt(1 - 2 * rho * z + z**2) + z - rho) / (1 - rho))
    
    factor = (F * K)**((beta - 1) / 2)
    return (alpha / factor) * (z / x_z)

def objective(params, market_vols, strikes, forwards, maturity, beta):
    alpha, rho, nu = params
    model_vols = [sabr_vol(alpha, rho, nu, F, K, maturity, beta) for F, K in zip(forwards, strikes)]
    return np.sum((model_vols - market_vols) ** 2)

# Example market data
market_vols = np.array([0.2, 0.21, 0.22])  # Example volatilities
strikes = np.array([100, 110, 120])        # Corresponding strike prices
forwards = np.array([0.5, 0.51, 0.56])       # Corresponding forward rates for each strike
maturity = 10                              # Maturity in years
beta_fixed = 0.5                           # Example fixed beta

# Initial guesses for parameters
initial_guess = [0.1, 0, 0.2]

# Boundaries for the parameters: alpha > 0, -1 < rho < 1, nu > 0
bounds = [(0, None), (-1, 1), (0, None)]

# Perform the minimization
result = minimize(objective, initial_guess, args=(market_vols, strikes, forwards, maturity, beta_fixed), bounds=bounds)
alpha_est, rho_est, nu_est = result.x

print("Estimated Parameters:", alpha_est*100, rho_est*100, nu_est*100)


Estimated Parameters: 1.4866551411398314 -85.8289872486463 3.1298854035054244


In [5]:
import numpy as np
import pandas as pd
from scipy.optimize import least_squares

# Correct implementation of the Black volatility calculation by SABR model
def blackvolbysabr(alpha, beta, rho, nu, settle, exercise_date, fwd_value, strike):
    # Placeholder for simplified Black-Scholes volatility computation
    # Typically involves complex SABR model formulas
    F = fwd_value
    K = strike
    tau = (exercise_date - settle).astype('timedelta64[D]').astype(int) / 365.25
    if F == K:
        vol = alpha
    else:
        z = nu / alpha * np.power(F * K, (1 - beta) / 2) * np.log(F / K)
        x = np.log(z / (1.0 - beta))
        vol = (alpha / (np.power(F * K, (1 - beta) / 2) * (1 + (1 - beta)**2 / 24 * np.log(F / K)**2 + (1 - beta)**4 / 1920 * np.log(F / K)**4))) * \
             (z / x) * (1 + ((1 - beta)**2 / 24 * alpha**2 / (F * K)**(1 - beta) + 1 / 4 * (rho * beta * nu * alpha) / (F * K)**((1 - beta) / 2) + (2 - 3 * rho**2) / 24 * nu**2) * tau)
    return vol

# Refactor function definitions outside the loop
def alpharoots(rho, nu, beta, years_to_exercise, current_forward_value, atm_volatility):
    coefficients = [
        (1 - beta)**2 * years_to_exercise / 24 / current_forward_value**(2 - 2 * beta),
        rho * beta * nu * years_to_exercise / 4 / current_forward_value**(1 - beta),
        1 + (2 - 3 * rho**2) * nu**2 * years_to_exercise / 24,
        -atm_volatility * current_forward_value**(1 - beta)
    ]
    return np.roots(coefficients)

def atmVol2SabrAlpha(rho, nu, beta, years_to_exercise, current_forward_value, atm_volatility):
    roots_arr = alpharoots(rho, nu, beta, years_to_exercise, current_forward_value, atm_volatility)
    real_roots = [x for x in roots_arr if x.imag == 0 and x.real > 0]
    if real_roots:
        return min(real_roots, key=lambda x: abs(x.real)).real
    return min(roots_arr, key=lambda x: abs(x)).real

def objFun(X, k, beta, years_to_exercise, current_forward_value, market_volatilities, settle, exercise_dates, market_strikes):
    rho, nu = X
    alpha = atmVol2SabrAlpha(rho, nu, beta, years_to_exercise, current_forward_value, ATMVolatilities[k])
    return market_volatilities[:, k] - blackvolbysabr(alpha, beta, rho, nu, settle, exercise_dates[k], current_forward_value, market_strikes[:, k])

# Placeholder for the actual data setup
NumMaturities = 8
YearsToExercise = np.array([1, 2, 3, 4, 5, 6, 7, 8])
CurrentForwardValues = np.linspace(100, 102, NumMaturities)
ATMVolatilities = np.linspace(0.2, 0.22, NumMaturities)
MarketVolatilities = np.random.normal(0.05, 0.06, (10, NumMaturities))
Settle = np.datetime64('today')
ExerciseDates = np.array([np.datetime64('today') + np.timedelta64(i, 'D') for i in range(NumMaturities)])
MarketStrikes = np.random.uniform(100, 200, (10, NumMaturities))

Beta = 0.5
Betas = np.repeat(Beta, NumMaturities)
Alphas = np.zeros(NumMaturities)
Rhos = np.zeros(NumMaturities)
Nus = np.zeros(NumMaturities)

# Calibration loop
for k in range(NumMaturities):
    result = least_squares(objFun, [0, 0.5], args=(k, Beta, YearsToExercise[k], CurrentForwardValues[k], MarketVolatilities, Settle, ExerciseDates, MarketStrikes), bounds=([-1, 0], [1, np.inf]), verbose=0)
    rho, nu = result.x
    alpha = atmVol2SabrAlpha


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [6]:
import numpy as np
import pandas as pd
from scipy.optimize import least_squares

def blackvolbysabr(alpha, beta, rho, nu, settle, exercise_date, fwd_value, strike):
    # Placeholder function, should be replaced with the actual implementation
    return np.log(alpha / fwd_value)  # Simplified placeholder

# Define the data from the image
vol_data = {
    'Tenor': ['1Y', '2Y', '3Y', '5Y', '7Y', '10Y', '12Y', '15Y', '20Y', '30Y'],
    'ATM': [0.539328, 0.533911, 0.521010, 0.506140, 0.495310, 0.483150, 0.467810, 0.452470, 0.435590, 0.417570]
}
vol_df = pd.DataFrame(vol_data)
vol_df.set_index('Tenor', inplace=True)

# Variables setup
NumMaturities = len(vol_df)
YearsToExercise = np.array([1, 2, 3, 5, 7, 10, 12, 15, 20, 30]) * 12  # Convert years to months
CurrentForwardValues = np.linspace(100, 200, NumMaturities)  # Simulated forward values
ATMVolatilities = vol_df['ATM'].values  # ATM volatilities from the data

Beta = 0.5
Betas = np.repeat(Beta, NumMaturities)
Alphas = np.zeros(NumMaturities)
Rhos = np.zeros(NumMaturities)
Nus = np.zeros(NumMaturities)

Settle = np.datetime64('today')
ExerciseDates = [Settle + np.timedelta64(month, 'M') for month in YearsToExercise]

def alpharoots(k, rho, nu):
    coefficients = [
        (1 - Beta)**2 * YearsToExercise[k] / 24 / CurrentForwardValues[k]**(2 - 2 * Beta),
        rho * Beta * nu * YearsToExercise[k] / 4 / CurrentForwardValues[k]**(1 - Beta),
        1 + (2 - 3 * rho**2) * nu**2 * YearsToExercise[k] / 24,
        -ATMVolatilities[k] * CurrentForwardValues[k]**(1 - Beta)
    ]
    return np.roots(coefficients)

def atmVol2SabrAlpha(k, rho, nu):
    roots_arr = alpharoots(k, rho, nu)
    real_roots = [x for x in roots_arr if x.imag == 0 and x.real > 0]
    if real_roots:
        return min(real_roots).real
    return min(roots_arr, key=abs).real

for k in range(NumMaturities):
    def objFun(X):
        rho, nu = X
        alpha = atmVol2SabrAlpha(k, rho, nu)
        return [ATMVolatilities[k] - blackvolbysabr(alpha, Beta, rho, nu, Settle, ExerciseDates[k], CurrentForwardValues[k], CurrentForwardValues[k])]

    result = least_squares(objFun, [0, 0.5], bounds=([-1, 0], [1, np.inf]), verbose=0)
    rho, nu = result.x
    Alphas[k] = atmVol2SabrAlpha(k, rho, nu)
    Rhos[k] = rho
    Nus[k] = nu

# Create a DataFrame to store the calibrated parameters
CalibratedParameters = np.column_stack((Alphas, Betas, Rhos, Nus))
CalibratedParameters = pd.DataFrame(CalibratedParameters, index=vol_df.index, columns=['Alpha', 'Beta', 'Rho', 'Nu'])
print(CalibratedParameters)


UFuncTypeError: Cannot cast ufunc 'add' input 1 from dtype('<m8[M]') to dtype('<m8[D]') with casting rule 'same_kind'

In [9]:
import numpy as np
import pandas as pd
from scipy.optimize import least_squares

def blackvolbysabr(alpha, beta, rho, nu, settle, exercise_date, fwd_value, strike):
    # Placeholder function, should be replaced with the actual implementation
    return np.log(alpha / fwd_value)  # Simplified placeholder

# Define the data from the image
vol_data = {
    'Tenor': ['1Y', '2Y', '3Y', '5Y', '7Y', '10Y', '12Y', '15Y', '20Y', '30Y'],
    'ATM': [0.539328, 0.533911, 0.521010, 0.506140, 0.495310, 0.483150, 0.467810, 0.452470, 0.435590, 0.417570]
}
vol_df = pd.DataFrame(vol_data)
vol_df.set_index('Tenor', inplace=True)

# Variables setup
NumMaturities = len(vol_df)
YearsToExercise = np.array([1, 2, 3, 5, 7, 10, 12, 15, 20, 30])  # Years until exercise for each tenor
CurrentForwardValues = np.linspace(0.2, 0.3, NumMaturities)  # Simulated forward values
ATMVolatilities = vol_df['ATM'].values  # ATM volatilities from the data

Beta = 0.5
Betas = np.repeat(Beta, NumMaturities)
Alphas = np.zeros(NumMaturities)
Rhos = np.zeros(NumMaturities)
Nus = np.zeros(NumMaturities)

Settle = np.datetime64('today')
# Convert years to days directly using 365.25 to account for leap years
ExerciseDates = [Settle + np.timedelta64(int(365.25 * year), 'D') for year in YearsToExercise]

def alpharoots(k, rho, nu):
    coefficients = [
        (1 - Beta)**2 * YearsToExercise[k] / 24 / CurrentForwardValues[k]**(2 - 2 * Beta),
        rho * Beta * nu * YearsToExercise[k] / 4 / CurrentForwardValues[k]**(1 - Beta),
        1 + (2 - 3 * rho**2) * nu**2 * YearsToExercise[k] / 24,
        -ATMVolatilities[k] * CurrentForwardValues[k]**(1 - Beta)
    ]
    return np.roots(coefficients)

def atmVol2SabrAlpha(k, rho, nu):
    roots_arr = alpharoots(k, rho, nu)
    real_roots = [x for x in roots_arr if x.imag == 0 and x.real > 0]
    if real_roots:
        return min(real_roots).real
    return min(roots_arr, key=abs).real

for k in range(NumMaturities):
    def objFun(X):
        rho, nu = X
        alpha = atmVol2SabrAlpha(k, rho, nu)
        return [ATMVolatilities[k] - blackvolbysabr(alpha, Beta, rho, nu, Settle, ExerciseDates[k], CurrentForwardValues[k], CurrentForwardValues[k])]

    result = least_squares(objFun, [0, 0.5], bounds=([-1, 0], [1, np.inf]), verbose=0)
    rho, nu = result.x
    Alphas[k] = atmVol2SabrAlpha(k, rho, nu)
    Rhos[k] = rho
    Nus[k] = nu

# Create a DataFrame to store the calibrated parameters
CalibratedParameters = np.column_stack((Alphas, Betas, Rhos, Nus))
CalibratedParameters = pd.DataFrame(CalibratedParameters, index=vol_df.index, columns=['Alpha', 'Beta', 'Rho', 'Nu'])
print(CalibratedParameters)


          Alpha  Beta       Rho        Nu
Tenor                                    
1Y     0.342971   0.5 -0.922527  2.182896
2Y     0.360069   0.5 -0.827019  1.944981
3Y     0.374162   0.5 -0.793424  1.710904
5Y     0.387071   0.5 -0.842740  0.899799
7Y     0.401135   0.5 -0.805268  0.782377
10Y    0.414300   0.5 -0.750617  0.709189
12Y    0.425732   0.5 -0.735439  0.659192
15Y    0.436720   0.5 -0.722981  0.580667
20Y    0.446586   0.5 -0.708708  0.487117
30Y    0.455480   0.5 -0.669014  0.407666
